Speed Detection and Car tracking

In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

try :
    from tracker import *
except ImportError :
    print('Import not done')

In [2]:
import ultralytics
import cv2
import pandas as pd
from ultralytics import YOLO


In [3]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0 


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [4]:
model = YOLO('yolov8s.pt')

In [5]:
class_list = ['person', 'bicycle', 'car','motorcycle', 'airplane', 'bus',
              'train', 'truck', 'traffic light']

# class_list = ['car','truck']

In [6]:
tracker = Tracker()
count = 0

In [7]:
down = {}

In [8]:
cap = cv2.VideoCapture('C:\\Users\\Aman\\Desktop\\thesisproject\\video\\highway (1).mp4')

In [ ]:
# Open the video file
cap = cv2.VideoCapture('C:\\Users\\Aman\\Desktop\\thesisproject\\video\\highway (1).mp4')

# Get the frames per second (FPS) of the video
fps = cap.get(cv2.CAP_PROP_FPS)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    result = model.predict(frame)
    a = result[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        x2 = int(row[2])
        y1 = int(row[1])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        red_line_y = 198
        blue_line_y = 268
        offset = 7

        # Condition for red line
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = cy
            if id in down:
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    text_color = (255, 255, 255)
    red_color = (0, 0, 255)
    blue_color = (255, 0, 0)
    green_color = (0, 255, 0)

    cv2.line(frame, (172, 198), (774, 198), red_color, 3)
    cv2.putText(frame, ('red line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.line(frame, (8, 268), (927, 268), blue_color, 3)
    cv2.putText(frame, ('blue line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('frames', frame)

    # Wait for the appropriate time based on FPS (frame delay in milliseconds)
    delay = int(1000 / fps)  # Delay per frame in milliseconds
    if cv2.waitKey(delay) & 0xFF == 27:  # ESC key to exit
        break

cap.release()
cv2.destroyAllWindows()



0: 320x640 8 cars, 2 trucks, 367.1ms
Speed: 6.2ms preprocess, 367.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 342.4ms
Speed: 0.0ms preprocess, 342.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 469.5ms
Speed: 0.0ms preprocess, 469.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 336.8ms
Speed: 0.0ms preprocess, 336.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 424.6ms
Speed: 0.0ms preprocess, 424.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 588.3ms
Speed: 5.8ms preprocess, 588.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 377.2ms
Speed: 3.7ms preprocess, 377.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 305.5ms
Speed: 8.0

IndexError: list index out of range

: 

In [9]:
import cv2
import numpy as np

img = np.zeros((100, 100, 3), dtype=np.uint8)
cv2.imshow("Test Window", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
